In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import punkt


In [9]:
import re
nltk.download('stopwords')


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
import threading
import os
import json

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_csv('Input.csv')

In [3]:
error_urls = []
error_ids = []
def article(url,url_id,fpath):
  try:
    ht = requests.get(url).text
    soup = bs(ht,'lxml')
    div = soup.find('div',class_='td-post-content')
    h1 = soup.find('h1')
    title = h1.text
    arti = div.text
    if div==None or arti==None:
      print(url)
    text = title + '.' + arti
    with open(f'{fpath+str(url_id)}.txt','w') as fp:
      fp.write(text)



  except:
    global error_urls
    global error_ids
    print(url)
    error_urls.append(url)
    error_ids.append(url_id)





article('https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/',1,'sample_data/Articles/')

https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/


In [5]:
def make_set(fpath):
  s = set()
  with open(fpath,'r') as fp:
    content = fp.read()
  words = content.split('\n')
  for i in words:
    s.add(i.lower())
  s.discard('')
  return s




In [6]:
def make_set2(fpath):
    ss = set()
    with open(fpath,'r') as f:
     content2 = f.read()
    word_raw = content2.split('\n')
    for ii in word_raw:
      if '|' in ii:
        ss.add(ii.split('|')[0].rstrip().lower())
      else:
        ss.add(ii.lower())
    ss.discard('')
    return ss


In [21]:
os.mkdir('articles')

In [7]:

with open('positive.json','r') as jp:
  pos = json.load(jp)
  pos = set(pos)
with open('negative.json','r') as jp1:
  neg = json.load(jp1)
  neg = set(neg)
with open('stop_words.json','r') as jp2:
  stop = json.load(jp2)
  stop_words = set(stop)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
import string

def rem_punkt(text):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)

    # Use the translation table to remove punctuation from the text
    text_without_punctuation = text.translate(translator)

    return text_without_punctuation



In [27]:
def is_num(s):
    try:
        float_num = float(s)
        return True
    except ValueError:
        return False



def sentiment_analysis(content,avoid_numbers = False):

  pattern = re.compile(r'[!#$%&()*+,./:;<=>?\\@\[\]_{|}~]')
  sentences = nltk.sent_tokenize(content)
  pos_count = 0
  neg_count = 0
  word_count = 0
  for s in sentences:
    sentence = pattern.sub(' ',s)
    word_list = sentence.split()
    for word in word_list:
      if word.lower() in stop_words:
        continue
      elif avoid_numbers and is_num(word):
        continue
      elif word.lower() in pos:
        pos_count += 1
        word_count +=1
      elif word.lower() in neg:
        neg_count += 1
        word_count += 1
      else:
        word_count += 1
  polarity_score = (pos_count-neg_count)/((pos_count+neg_count)+0.000001)
  subjectivity_score = (pos_count+neg_count)/(word_count + 0.000001)
  return [pos_count,neg_count,polarity_score,subjectivity_score]



In [18]:
def syllable(s):
  count = 0
  vowels = ['a','e','i','o','u']
  vowels = set(vowels)
  for i in s:
    if i in vowels:
      count += 1
  if len(s)>1:
    if s[-2]=='e':
      if s[-1]=='d' or s[-1]=='s':
        count = count - 1
  return count


def sentence_analysis(content):
  pronoun_count = 0
  word_count = 0
  complex_count = 0
  syllable_count = 0
  letter_count = 0
  personal_pronouns = ['I','me','Me','Us','us','Ours','ours','we','We']
  pattern = re.compile(r'[!#$&()*+,./:;<=>?\\@\[\]{|}~]')
  sentences = nltk.sent_tokenize(content)
  total_sentences = len(sentences)
  for sent in sentences:
    sent = pattern.sub(' ',sent)
    words = sent.split()
    for word in words:
      if word.lower() in stopwords.words('english'):
        continue
      else:
        word = rem_punkt(word)
        if word!='':
          word_count += 1
          letter_count += len(word)
          syllab = syllable(word.lower())
          if syllab>2:
            complex_count +=1
          syllable_count += syllab
          if word in personal_pronouns:
            pronoun_count += 1
  avg_sentence_len = word_count/total_sentences
  per_complex_words = (complex_count/word_count)*100
  avg_word_len = word_count/letter_count
  fog_index = (avg_sentence_len+per_complex_words)*0.4
  word_per_sent = word_count/total_sentences
  syllable_per_word = syllable_count/word_count
  return [avg_sentence_len, per_complex_words, fog_index, word_per_sent, complex_count, word_count, syllable_per_word,pronoun_count,avg_word_len]










In [23]:
error_urls1 = []
error_ids1 = []
def final(i):
  global df2
  try:
    url = df2['URL'][i]
    url_id = df2['URL_ID'][i]
    ht = requests.get(url).text
    soup = bs(ht,'lxml')
    div = soup.find('div',class_='td-post-content')
    h1 = soup.find('h1')
    title = h1.text
    arti = div.text
    if div==None or arti==None:
      print(url)
    text = title + '.' + arti
    fpath = './articles/'
    with open(f'{fpath+str(url_id)}.txt','w') as fp:
      fp.write(text)
    pos_count,neg_count,polarity_score,subjectivity_score = sentiment_analysis(text)
    df2.loc[i,'POSITIVE SCORE'] = pos_count
    df2.loc[i,'NEGATIVE SCORE'] = neg_count
    df2.loc[i,'POLARITY SCORE'] = polarity_score
    df2.loc[i,'SUBJECTIVITY SCORE'] = subjectivity_score
    anslis = sentence_analysis(text)
    df2.loc[i,'AVG SENTENCE LENGTH'] = anslis[0]
    df2.loc[i,'PERCENTAGE OF COMPLEX WORDS'] = anslis[1]
    df2.loc[i,'FOG INDEX'] = anslis[2]
    df2.loc[i,'AVG NUMBER OF WORDS PER SENTENCE'] = anslis[3]
    df2.loc[i,'COMPLEX WORD COUNT'] = anslis[4]
    df2.loc[i,'WORD COUNT'] = anslis[5]
    df2.loc[i,'SYLLABLE PER WORD'] = anslis[6]
    df2.loc[i,'PERSONAL PRONOUNS'] = anslis[7]
    df2.loc[i,'AVG WORD LENGTH'] = anslis[8]


  except:
    global error_urls1
    global error_ids1
    print(url)
    error_urls1.append(url)
    error_ids1.append(url_id)


In [22]:
df2 = pd.read_csv('output_struct.csv')
df2.columns

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [29]:
for oo in range(len(df2)):
  final(oo)
df2.to_csv('output.csv')

836
285
521
644
644
575
558
836
209
511
177
589
276
471
466
552
546
966
574
827
651
809
575
335
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
329
933
1011
582
603
658
746
249
711
458
363
147
https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
655
319
761
325
194
548
328
150
87
344
81
616
236
455
809
704
627
318
499
168
722
405
448
579
710
431
660
365
588
782
1619
769
815
68
452
685
801
486
911
449
506
464
673
73
492
335
544
410
215
279
527
240
507
465
516
588
697
923
357
929
510
191
1098
549
796
795
1046
818
391
786
408
354
831
888
635
523


In [30]:
import shutil

folder_to_download = 'articles'
output_zip_file = 'articles.zip'

shutil.make_archive(output_zip_file.split('.')[0], 'zip', folder_to_download)


'/content/articles.zip'

In [28]:
print(error_ids1,error_urls1)

[] []
